In [1]:
import os 
import pandas as pd
import numpy as np
import psycopg2


In [2]:
#find csv files in current working directory and kept it in a list
csv_files=[]
for file in os.listdir(os.getcwd()):
    if file.endswith(".csv"):
        csv_files.append(file)


In [3]:
#make new directory containing only csv files


#this code is wrong
for file in csv_files:
    #mkdir=f"mkdir {file}"
    #os.system(mkdir)
    pass


In [4]:
#make new directory containing only csv files
dataset_dir= "datasets"
#create bash command to make new directory 
#   mkdir dataset_dir
#error handling incase the file already exists in the directory and shows error
try:
    mkdir= 'mkdir {0}'.format(dataset_dir)
    os.system(mkdir)
except:
    pass

mkdir: datasets: File exists


In [5]:
#move the csv files into the new directory i.e datasets
#using bash command
#   mv filename directory
for file in csv_files:
    mv_file = "mv '{0}' {1}".format(file, dataset_dir)
    os.system(mv_file)
    print(mv_file)
    

mv 'COVID_vaccination.csv' datasets
mv 'COVID_deaths.csv' datasets


In [6]:
#loading the csv file in pandas dataframe using a dictionary
csv_datasets={}
file_path=os.getcwd()+'/'+dataset_dir+'/'
for file in csv_files:
    csv_datasets[file]= pd.read_csv(file_path+file)
    

In [7]:
#cleaning data heads
#replacing spaces with _ and removing speacial symbols like $# etc
for file in csv_files:
    dataframe= csv_datasets[file]
    clean_tbl_name=file.lower().\
        replace(" ","_").replace("#","").replace("$","").replace("~","").replace(".","")
    clean_tbl_name= '{0}'.format(clean_tbl_name.split(".")[0])
    
    columns=[]
    for x in dataframe.columns:
        if x.endswith(' '):
            columns.append(x.strip(" "))
        else:
            columns.append(x)
    dataframe.columns=columns
    
    dataframe.columns=[x.lower()\
        .replace(" ","_").replace("#","").replace("$","").replace("~","").replace(".","") for x in dataframe.columns]
#replacement dictionary that maps pandas datatype to SQL datatype
    replacements={
        'object' : 'varchar',
        'float64':'float',
        'int64' : 'bigint',
        'datetime64' : 'timestamp',
        'timedelta64[ns]' : 'varchar'
    }

#table schema
    col_str= ",".join("{} {}".format(n,d) for (n,d) in zip(dataframe.columns, dataframe.dtypes.replace(replacements)))
#    print(col_str)
    con_string="host=localhost \
                dbname='COVID_Dataset'\
                user='postgres'\
                password='@ imp9841 @'"
    with psycopg2.connect(con_string) as conn:
        with conn.cursor() as cursor:   
            print("Opened Database Succesfully")
#drop duplicated tables
            cursor.execute("DROP TABLE IF EXISTS {};".format(clean_tbl_name))
            print(clean_tbl_name)
#create new tables
            cursor.execute("CREATE TABLE IF NOT EXISTS {} ({})".format(clean_tbl_name,col_str))
    

#insert value to table
#save df to csv
            dataframe.to_csv(file,header=dataframe.columns, index=False, encoding='utf-8')
#open csv file, save it as an obj
            myfile= open(file)
            print('file op i mem')
#grant permission to multiple users to access my database
            cursor.execute('grant select on table %s to public' % clean_tbl_name)
    
#upload to db
            sqlstatement= """
                COPY %s FROM STDIN WITH
                CSV
                HEADER
                DELIMITER AS ','
       """
            cursor.copy_expert(sql=sqlstatement % clean_tbl_name, file=myfile)
            print("file copied to db")

    
        
    conn.close()




Opened Database Succesfully
covid_vaccinationcsv
file op i mem
file copied to db
Opened Database Succesfully
covid_deathscsv
file op i mem
file copied to db
